In [1]:
import chex
import jax.numpy as jnp
from jax.scipy.signal import convolve2d, correlate2d
from enum import Enum
from typing import Optional, List

In [20]:
GOMOKU_BOARD_SIZE = 20

class Player(Enum):
    PLAYER_1 = 1
    PLAYER_2 = 2
    # A methord to get the other
    def other(self):
        return Player.PLAYER_1 if self == Player.PLAYER_2 else Player.PLAYER_2
        
@chex.dataclass(frozen=True)
class Board:
    """
    This holds the current state of the board which is a 2D array of size GOMOKU_BOARD_SIZE x GOMOKU_BOARD_SIZE.
    Moves of the players are represented by 1 and 2 respectively.
    """
    # 0 for empty, 1 for player 1, 2 for player 2.
    state: chex.ArrayDevice = jnp.zeros(
        (GOMOKU_BOARD_SIZE, GOMOKU_BOARD_SIZE), dtype=jnp.int32)
    player: Player = Player.PLAYER_1  # The player who is making the next move.
    # An equals method for testing.

    def __eq__(self, other):
        return jnp.all(self.state == other.state) and self.player == other.player


@chex.dataclass(frozen=True)
class GameOutcome:
    """
    This holds the outcome of a game.
    """
    class WinType(Enum):
        ROW = 0
        COL = 1
        DIAG = 2
        ALT_DIAG = 3
        DRAW = 4
    winner: Optional[Player] = None  # The winner of the game.
    row_loc: int = -1  # Row location of the winning move.
    col_loc: int = -1  # Column location of the winning move.
    win_type: Optional[WinType] = None  # Type of the winning move.


@chex.dataclass(frozen=True)
class StepOutput:
    """
    This holds the output of the step function.
    """
    board: Board
    reward: int
    done: bool
    # An equals method for testing.

    def __eq__(self, other):
        return self.board == other.board and self.reward == other.reward and self.done == other.done


class GomokuEnv:
    @classmethod
    def action_to_index(cls, action: int) -> (int, int):
        """
        This function takes an action and returns the corresponding 2D index.
        param action: An integer between 0 and 399.
        """
        assert action >= 0 and action < GOMOKU_BOARD_SIZE * GOMOKU_BOARD_SIZE
        row = action // GOMOKU_BOARD_SIZE
        col = action % GOMOKU_BOARD_SIZE
        return (row, col)

    @classmethod
    def kernels(cls, kernel_size=5) -> jnp.ndarray:
        """
        This function returns the kernels for checking if there is a winner.
        """
        row_kernel = jnp.zeros((kernel_size, kernel_size))
        # set the first row to 1.
        row_kernel = row_kernel.at[0, :].set(1)
        col_kernel = row_kernel.T
        diag_kernel = jnp.eye(kernel_size)
        alt_diag_kernel = jnp.fliplr(diag_kernel)
        # All kernels stacked together.
        kernels = jnp.stack(
            [row_kernel, col_kernel, diag_kernel, alt_diag_kernel])
        return kernels

    @classmethod
    def __expanded_board(cls, board: Board) -> jnp.ndarray:
        """
        This function takes a board and returns an expanded version of it.
        The expanded board has one channel for each player.
        param board: A board object whose state needs to be expanded.
        """
        return jnp.stack([board.state == Player.PLAYER_1.value,
                          board.state == Player.PLAYER_2.value]).astype(jnp.int32)

    @classmethod
    def __win_locations(cls, board: Board) -> jnp.ndarray:
        """
        This function takes a board and finds the locations of the winning move.
        param board: A board object whose state needs to be expanded.
        returns: A 2D array of size Nx3 where N is the number of winning moves as computed by the location where the kernels were operated on.
        The three columns are the (kernel index per player, row index and column index). 
        0-3 kernel index corresponds to player 1 and 4-8 kernel index corresponds to player 2.
        """
        kernels = cls.kernels()
        board_expanded = cls.__expanded_board(board)
        # valid mode of convolution is a mode that does not extend the board.
        kernel_outpus = jnp.array([correlate2d(board_of_player, kernel, mode="valid")
                                   for board_of_player in board_expanded for kernel in kernels])
        # Check if there is a winner using argwhere for the value is equal to the number of pieces in a row.
        locations = jnp.argwhere(kernel_outpus == kernels.shape[1])
        return locations

    @classmethod
    def assert_valid(cls, board: Board) -> bool:
        """
        A helper function to check if a board is valid. This checks the following:
        1. The board is a 2D array of size 20x20.
        2. The board has only 0, Player.PLAYER_1.value and Player.PLAYER_2.value
        3. It does not have multiple winners.
        4. Each player played the same number of moves or one more move.
        Raise a unique error if any of the above conditions are not satisfied.
        """
        # Check the shape of the board.
        if board.state.shape != (GOMOKU_BOARD_SIZE, GOMOKU_BOARD_SIZE):
            raise ValueError(
                f"The board state is not of shape ({GOMOKU_BOARD_SIZE}, {GOMOKU_BOARD_SIZE}).")
        # Check that the board has only 0, 1 and 2.
        if not jnp.all(jnp.isin(board.state, jnp.array([0, Player.PLAYER_1.value, Player.PLAYER_2.value], dtype=jnp.int32))):
            raise ValueError(
                f"The board state has values other than 0, {Player.PLAYER_1.value} and {Player.PLAYER_2.value}.")
        # Check that the board does not have multiple winners.
        if cls.__win_locations(board).shape[0] > 1:
            raise ValueError("The board has multiple winners.")
        # Check that the number of moves played by each player is the same or one more.
        if jnp.sum(board.state == Player.PLAYER_1.value) - jnp.sum(board.state == Player.PLAYER_2.value) not in [0, 1]:
            raise ValueError(
                f"The number of moves played by each player is not the same or one more."
                f"Player 1: {jnp.sum(board.state == Player.PLAYER_1.value)},"
                f"Player 2: {jnp.sum(board.state == Player.PLAYER_2.value)}")
        return True

    @classmethod
    def outcome(cls, board: Board) -> GameOutcome:
        """
        A game is over if there is a winner or the board is full.
        To check for a winner, we need to check if there are 5 consecutive
        pieces of the same player in a row, column or diagonal. This will return
        the current game outcome. (Win, Lose, Draw, On-going)
        param board: A board object whose outcome needs to be checked.
        """
        locations = cls.__win_locations(board)
        # There can be at most one winner, otherwise the board is invalid.
        assert locations.shape[0] <= 1
        if locations.shape[0] > 0:
            # There is a winner.
            player = Player.PLAYER_1 if locations[0, 0].item(
            ) < 4 else Player.PLAYER_2
            win_type = GameOutcome.WinType(locations[0, 0].item() % 4)
            row_loc = locations[0, 1].item()
            # If the win type is ALT_DIAG, we need to add 4 to the column location. because the alt diag kernel is flipped.
            col_loc = 4 + \
                locations[0, 2].item(
                ) if win_type == GameOutcome.WinType.ALT_DIAG else locations[0, 2].item()
            return GameOutcome(winner=player, row_loc=row_loc, col_loc=col_loc, win_type=win_type)
        # If we reached here without finding a winner and the board is full it's a draw
        if jnp.all(board.state != 0):
            return GameOutcome(winner=None, row_loc=-1, col_loc=-1, win_type=GameOutcome.WinType.DRAW)
        # There is no winner and we can continue the game
        # The game can continue.
        return GameOutcome(winner=None, row_loc=-1, col_loc=-1, win_type=None)

    @classmethod
    def step(cls, board: Board, action: int, player: Player) -> StepOutput:
        """
        This function takes a board state and an action and returns the new board state.
        This function also checks if the game is over, and returns the reward.
        param board: A valid board object.
        param action: An integer between 0 and 399.
        param player: The player who is making the move.
        """
        # Action can be anything from 0 to 399.
        assert action >= 0 and action < GOMOKU_BOARD_SIZE * GOMOKU_BOARD_SIZE
        # Check that the game is not over yet.
        if cls.outcome(board).win_type is not None:
            return StepOutput(board=board, reward=-1, done=True)  # The game is over.

        # We need to convert it to a 2D index.
        row, col = cls.action_to_index(action)
        # Check if the action is valid.
        if board.state[row, col] != 0:
            return StepOutput(board=board, reward=-1, done=True)
        if player != board.player:
            raise ValueError("Requested player is not the current player!")
        # Update the board state.
        new_state = Board(state=board.state.at[row, col].set(
            player.value), player=player.other())
        # Check if the game is over.
        outcome = cls.outcome(new_state)
        reward = 1 if outcome.winner == player else -1 if outcome.winner == player.other() else 0
        return StepOutput(board=new_state, reward=reward, done=outcome.win_type is not None)


In [21]:
# A class to produce gomoku scenarios easily for unit testing.

@chex.dataclass(frozen=True)
class GomokuStroke:
    class StrokeType(Enum):
        ROW = 0
        COL = 1
        DIAG = 2
        ALT_DIAG = 3
    """Describes a stroke to paint on the board."""
    player: int
    row: int
    col: int
    stroke_type: StrokeType
    length: int

class GomokuScenario:
    """
    This class is used to produce scenarios for unit testing. Give it a list of strokes and it will produce a board.
    """
    @classmethod
    def make_scenario(cls, strokes: List[GomokuStroke], end_player: Player):
        """
        This function takes a list of strokes and produces a board.
        The player field in the returned board is set to the end_player.
        """
        state = jnp.zeros((GOMOKU_BOARD_SIZE, GOMOKU_BOARD_SIZE), dtype=jnp.int32)
        for stroke in strokes:
            kernels = GomokuEnv.kernels(kernel_size=stroke.length)
            kernel = kernels[stroke.stroke_type.value]
            starting_column = stroke.col if stroke.stroke_type != GomokuStroke.StrokeType.ALT_DIAG else stroke.col - stroke.length + 1
            state = state.at[stroke.row:stroke.row+stroke.length, starting_column: starting_column+stroke.length].set(
                jnp.where(kernel == 1, stroke.player.value, state[stroke.row:stroke.row+stroke.length, starting_column: starting_column+stroke.length])
                )
        return Board(state=state, player=end_player)

# Unit Tests

In [22]:
import unittest
import numpy as np

In [23]:
# Example usage and unit test
class TestGomokuScenarioMaker(unittest.TestCase):
    def test_scenario_with_all_stroke_types(self):
        strokes = [
            GomokuStroke(player=Player.PLAYER_1, row=0, col=0, stroke_type=GomokuStroke.StrokeType.ROW, length=5),
            GomokuStroke(player=Player.PLAYER_2, row=1, col=0, stroke_type=GomokuStroke.StrokeType.COL, length=2),
            GomokuStroke(player=Player.PLAYER_1, row=3, col=0, stroke_type=GomokuStroke.StrokeType.DIAG, length=3),
            GomokuStroke(player=Player.PLAYER_2, row=4, col=4, stroke_type=GomokuStroke.StrokeType.ALT_DIAG, length=5),
        ]
        scenario = GomokuScenario.make_scenario(strokes, Player.PLAYER_1)
        expected_array = jnp.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
        
        np.testing.assert_array_equal(scenario.state, expected_array)
        self.assertEqual(scenario.player, Player.PLAYER_1)
    
    def test_scenario_with_one_alt_diag(self):
        scenario = GomokuScenario.make_scenario([
            GomokuStroke(player=Player.PLAYER_1, row=0, col=6, stroke_type=GomokuStroke.StrokeType.ALT_DIAG, length=5),
            GomokuStroke(player=Player.PLAYER_2, row=6, col=0, stroke_type=GomokuStroke.StrokeType.COL, length=4),
        ], Player.PLAYER_1)

        expected_first_12_by_12 = jnp.array([
            [0,0,0,0,0,0,1,0,0,0,0,0],
            [0,0,0,0,0,1,0,0,0,0,0,0],
            [0,0,0,0,1,0,0,0,0,0,0,0],
            [0,0,0,1,0,0,0,0,0,0,0,0],
            [0,0,1,0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0,0,0,0],
            [2,0,0,0,0,0,0,0,0,0,0,0],
            [2,0,0,0,0,0,0,0,0,0,0,0],
            [2,0,0,0,0,0,0,0,0,0,0,0],
            [2,0,0,0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0,0,0,0]])
        
        np.testing.assert_array_equal(scenario.state[:12, :12], expected_first_12_by_12)


In [34]:
class TestGomokuEnv(unittest.TestCase):
    # Test the __action_to_index function.
    def test_action_to_index(self):
        self.assertEqual(GomokuEnv.action_to_index(0), (0, 0))
        self.assertEqual(GomokuEnv.action_to_index(399), (19, 19))
        self.assertEqual(GomokuEnv.action_to_index(200), (10, 0))
        self.assertEqual(GomokuEnv.action_to_index(399), (19, 19))
        with self.assertRaises(AssertionError):
            GomokuEnv.action_to_index(-1)
        with self.assertRaises(AssertionError):
            GomokuEnv.action_to_index(400)
    
    # Test that the kernels are correct.
    def test_kernels(self):
        kernels = GomokuEnv.kernels()
        self.assertEqual(kernels.shape, (4, 5, 5))
        self.assertTrue(jnp.all(kernels[0] == jnp.array([[1, 1, 1, 1, 1],
                                                         [0, 0, 0, 0, 0],
                                                         [0, 0, 0, 0, 0],
                                                         [0, 0, 0, 0, 0],
                                                         [0, 0, 0, 0, 0]])))
        self.assertTrue(jnp.all(kernels[1] == jnp.array([[1, 0, 0, 0, 0],
                                                         [1, 0, 0, 0, 0],
                                                         [1, 0, 0, 0, 0],
                                                         [1, 0, 0, 0, 0],
                                                         [1, 0, 0, 0, 0]])))
        self.assertTrue(jnp.all(kernels[2] == jnp.array([[1, 0, 0, 0, 0],
                                                            [0, 1, 0, 0, 0],
                                                            [0, 0, 1, 0, 0],
                                                            [0, 0, 0, 1, 0],
                                                            [0, 0, 0, 0, 1]])))
        self.assertTrue(jnp.all(kernels[3] == jnp.array([[0, 0, 0, 0, 1],
                                                            [0, 0, 0, 1, 0],
                                                            [0, 0, 1, 0, 0],
                                                            [0, 1, 0, 0, 0],
                                                            [1, 0, 0, 0, 0]])))
    
    # Create a few boards that are invalid and check that assert_valid raises an error.
    def test_assert_invalid(self):
        # Test that the board is of the right shape.
        with self.assertRaises(ValueError):
            GomokuEnv.assert_valid(Board(state=jnp.zeros((10, 10), dtype=jnp.int32)))
        # Test that the board has only 0, 1 and 2.
        with self.assertRaises(ValueError):
            GomokuEnv.assert_valid(Board(state=312*jnp.ones((20, 20), dtype=jnp.int32)))
        # Test that the board has only one winner.
        with self.assertRaises(ValueError):
            GomokuEnv.assert_valid(GomokuScenario.make_scenario([
                GomokuStroke(player=Player.PLAYER_1, row=0, col=0, stroke_type=GomokuStroke.StrokeType.ROW, length=5),
                GomokuStroke(player=Player.PLAYER_2, row=1, col=0, stroke_type=GomokuStroke.StrokeType.COL, length=5),
            ], Player.PLAYER_1))  # This has two winners.
        # Test that the number of moves played by each player is the same or one more.
        with self.assertRaises(ValueError):
            GomokuEnv.assert_valid(GomokuScenario.make_scenario([
                GomokuStroke(player=Player.PLAYER_1, row=0, col=0, stroke_type=GomokuStroke.StrokeType.ROW, length=4),
                GomokuStroke(player=Player.PLAYER_2, row=1, col=0, stroke_type=GomokuStroke.StrokeType.COL, length=3),
                GomokuStroke(player=Player.PLAYER_1, row=2, col=0, stroke_type=GomokuStroke.StrokeType.COL, length=2),
            ], Player.PLAYER_1))
    
    # Create a few boards that are valid and check that assert_valid does not raise an error.
    def test_assert_valid(self):
        GomokuEnv.assert_valid(GomokuScenario.make_scenario([
            GomokuStroke(player=Player.PLAYER_1, row=0, col=0, stroke_type=GomokuStroke.StrokeType.ROW, length=5),
            GomokuStroke(player=Player.PLAYER_2, row=1, col=0, stroke_type=GomokuStroke.StrokeType.COL, length=4),
        ], Player.PLAYER_1))
        GomokuEnv.assert_valid(GomokuScenario.make_scenario([
            GomokuStroke(player=Player.PLAYER_1, row=0, col=0, stroke_type=GomokuStroke.StrokeType.ROW, length=4),
            GomokuStroke(player=Player.PLAYER_2, row=1, col=0, stroke_type=GomokuStroke.StrokeType.COL, length=4),
            GomokuStroke(player=Player.PLAYER_1, row=6, col=0, stroke_type=GomokuStroke.StrokeType.COL, length=1),
        ], Player.PLAYER_1))

    # Test that we can asses a winning outcome and correctly locate it in an alternate diagonal scenario
    def test_winning_outcome_alt_diag(self):
        outcome = GomokuEnv.outcome(GomokuScenario.make_scenario([
            GomokuStroke(player=Player.PLAYER_1, row=0, col=6, stroke_type=GomokuStroke.StrokeType.ALT_DIAG, length=5),
            GomokuStroke(player=Player.PLAYER_2, row=6, col=0, stroke_type=GomokuStroke.StrokeType.COL, length=4),
        ], Player.PLAYER_1))
        self.assertEqual(outcome.winner, Player.PLAYER_1)
        self.assertEqual(outcome.win_type, GameOutcome.WinType.ALT_DIAG)
        self.assertEqual(outcome.row_loc, 0)
        self.assertEqual(outcome.col_loc, 6)
    
    # Test that we can asses a winning outcome and correctly locate it in a row scenario
    def test_winning_outcome_row(self):
        outcome = GomokuEnv.outcome(GomokuScenario.make_scenario([
            GomokuStroke(player=Player.PLAYER_1, row=10, col=10, stroke_type=GomokuStroke.StrokeType.ROW, length=5),
            GomokuStroke(player=Player.PLAYER_2, row=6, col=0, stroke_type=GomokuStroke.StrokeType.COL, length=4),
        ], Player.PLAYER_1))
        self.assertEqual(outcome.winner, Player.PLAYER_1)
        self.assertEqual(outcome.win_type, GameOutcome.WinType.ROW)
        self.assertEqual(outcome.row_loc, 10)
        self.assertEqual(outcome.col_loc, 10)
    
    # Test that we can assess a ongoing game state
    def test_ongoing_outcome(self):
        # Create a random but valid game state
        outcome = GomokuEnv.outcome(GomokuScenario.make_scenario([
            GomokuStroke(player=Player.PLAYER_1, row=10, col=10, stroke_type=GomokuStroke.StrokeType.ROW, length=3),
            GomokuStroke(player=Player.PLAYER_2, row=6, col=0, stroke_type=GomokuStroke.StrokeType.COL, length=4),
        ], Player.PLAYER_1))
        self.assertEqual(outcome.winner, None)
        self.assertEqual(outcome.win_type, None)
        self.assertEqual(outcome.row_loc, -1)
        self.assertEqual(outcome.col_loc, -1)

    # Test that the step function works as expected in multiple scenarios
    def test_multiple_steps(self):
        # Create an empty board
        board = Board()
        output = GomokuEnv.step(board, 0, Player.PLAYER_1)
        expected_output = StepOutput(board=GomokuScenario.make_scenario([
            GomokuStroke(player=Player.PLAYER_1, row=0, col=0, stroke_type=GomokuStroke.StrokeType.ROW, length=1),
        ], Player.PLAYER_2), reward=0, done=False)
        self.assertEqual(output, expected_output)
        # Now try adding another move at the same location and see that the state is done and reward is -1
        abrupt_end_output = GomokuEnv.step(output.board, 0, Player.PLAYER_2)
        self.assertEqual(abrupt_end_output.reward, -1)
        self.assertEqual(abrupt_end_output.done, True)
        # Now let's play the game a few more steps
        output = GomokuEnv.step(output.board, 100, Player.PLAYER_2)
        output = GomokuEnv.step(output.board, 1, Player.PLAYER_1)
        output = GomokuEnv.step(output.board, 101, Player.PLAYER_2)
        output = GomokuEnv.step(output.board, 2, Player.PLAYER_1)
        output = GomokuEnv.step(output.board, 102, Player.PLAYER_2)
        output = GomokuEnv.step(output.board, 3, Player.PLAYER_1)
        output = GomokuEnv.step(output.board, 103, Player.PLAYER_2)
        output = GomokuEnv.step(output.board, 4, Player.PLAYER_1)
        expected_final_output = StepOutput(board=GomokuScenario.make_scenario([
            GomokuStroke(player=Player.PLAYER_1, row=0, col=0, stroke_type=GomokuStroke.StrokeType.ROW, length=5),
            GomokuStroke(player=Player.PLAYER_2, row=5, col=0, stroke_type=GomokuStroke.StrokeType.ROW, length=4),
        ], Player.PLAYER_2), reward=1, done=True)
        self.assertEqual(output, expected_final_output)

unittest.main(argv=[''], verbosity=2, exit=False)
                                                                


test_action_to_index (__main__.TestGomokuEnv) ... ok
test_assert_invalid (__main__.TestGomokuEnv) ... ok
test_assert_valid (__main__.TestGomokuEnv) ... ok
test_kernels (__main__.TestGomokuEnv) ... ok
test_multiple_steps (__main__.TestGomokuEnv) ... ok
test_ongoing_outcome (__main__.TestGomokuEnv) ... ok
test_winning_outcome_alt_diag (__main__.TestGomokuEnv) ... ok
test_winning_outcome_row (__main__.TestGomokuEnv) ... ok
test_scenario_with_all_stroke_types (__main__.TestGomokuScenarioMaker) ... ok
test_scenario_with_one_alt_diag (__main__.TestGomokuScenarioMaker) ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.401s

OK
